In [17]:
import numpy as np 
import pandas as pd 

import shapefile as sf
from pyproj import Proj
from censusgeocode import CensusGeocode

import datetime

%matplotlib inline

In [8]:
sl_reader = sf.Reader('shapefiles/Street_Light/Street_Light.shp')
sl_record = pd.DataFrame(sl_reader.records())
sl_shapes = sl_reader.shapes()

In [4]:
# Need to convert each Street Light location to latitude and longitude
sl_record['lon'] = None
sl_record['lat'] = None

In [9]:
p1 = Proj("+init=EPSG:2230", preserve_units=True)
print len(sl_record)
for i in range(0,len(sl_record)): 
    lon, lat = p1(sl_shapes[i].points[0][0], sl_shapes[i].points[0][1],inverse=True)
    sl_record.loc[i, 'lon'] = lon 
    sl_record.loc[i, 'lat'] = lat
    if (i % 500==0):
        print i, i, i 

48455
0 0 0
500 500 500
1000 1000 1000
1500 1500 1500
2000 2000 2000
2500 2500 2500
3000 3000 3000
3500 3500 3500
4000 4000 4000
4500 4500 4500
5000 5000 5000
5500 5500 5500
6000 6000 6000
6500 6500 6500
7000 7000 7000
7500 7500 7500
8000 8000 8000
8500 8500 8500
9000 9000 9000
9500 9500 9500
10000 10000 10000
10500 10500 10500
11000 11000 11000
11500 11500 11500
12000 12000 12000
12500 12500 12500
13000 13000 13000
13500 13500 13500
14000 14000 14000
14500 14500 14500
15000 15000 15000
15500 15500 15500
16000 16000 16000
16500 16500 16500
17000 17000 17000
17500 17500 17500
18000 18000 18000
18500 18500 18500
19000 19000 19000
19500 19500 19500
20000 20000 20000
20500 20500 20500
21000 21000 21000
21500 21500 21500
22000 22000 22000
22500 22500 22500
23000 23000 23000
23500 23500 23500
24000 24000 24000
24500 24500 24500
25000 25000 25000
25500 25500 25500
26000 26000 26000
26500 26500 26500
27000 27000 27000
27500 27500 27500
28000 28000 28000
28500 28500 28500
29000 29000 29000
2950

In [14]:
cg = CensusGeocode()

In [15]:
sl_record.loc[i, 'block'] = None
sl_record.loc[i, 'tract'] = None
sl_record.loc[i, 'geoid'] = None
sl_record.loc[i, 'blockgroup'] = None

In [19]:
start = datetime.datetime.now()

for i in range(0,10): 
    if ((sl_record.loc[i,'lon']!=0.0) 
        & (np.isnan(sl_record.loc[i,'lon'])==False)) :
        temp1 = cg.coordinates(x=sl_record.loc[i,'lon'], 
                              y=sl_record.loc[i, 'lat'])
        if (len(temp1[0]['2010 Census Blocks'])>0):
            if ('status' not in temp1[0]['2010 Census Blocks'][0].keys()[0]): 
                temp = temp1[0]['2010 Census Blocks'][0]
                if (temp is not None):
                    sl_record.loc[i, 'block'] = temp['BLOCK']
                    sl_record.loc[i, 'tract'] = temp['TRACT']
                    sl_record.loc[i, 'geoid'] = temp['GEOID']
                    sl_record.loc[i, 'blockgroup'] = temp['BLKGRP']

                if (i % 100 == 0): 
                    finish = datetime.datetime.now()
                    print i, finish-start
                    start = datetime.datetime.now()



0 0:00:01.840060


In [20]:
sl_record[['blockgroup', 'tract']]

,blockgroup,tract
0,3,007600
1,3,007600
2,2,005600
3,2,005600
4,2,003401
5,2,003401
6,2,003401
7,2,003401
8,1,000600
9,1,000600
